In [70]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize
from ortools.sat.python import cp_model
from pprint import pprint

In [71]:
df = pd.read_csv("combined_data.csv")

In [72]:
df.head()

,date,hour,staff_id,name,level,status,predicted_sales,pred_sale_per_hour,max_cost,salary
0,2026-01-02,9,1,福田 美加子,3,part-time,205241,10262.05,2565,1350
1,2026-01-02,10,1,福田 美加子,3,part-time,205241,10262.05,2565,1350
2,2026-01-02,11,1,福田 美加子,3,part-time,205241,30786.15,7696,1350
3,2026-01-02,12,1,福田 美加子,3,part-time,205241,51310.25,12827,1350
4,2026-01-02,13,1,福田 美加子,3,part-time,205241,51310.25,12827,1350


In [73]:
model = cp_model.CpModel()

In [74]:
"""
data_set から .groupを使うことで特定のcolumn をkeys 
で　その特定のindex をvalues として取得します。
.key()は通常のpython のdictからkeyだけを取ります
"""
time_strip = list(df.groupby(["date", "hour"]).groups.keys())

In [75]:
"var 定義"
work = {}
for _, row in df.iterrows():
    s = row["staff_id"]
    d = row["date"]
    h = row["hour"]
    work[s, d, h] = model.NewBoolVar(f"work_{s}_{d}_{h}")


In [76]:
cost = {}
max_cost = {}
pred_sales = {}
for _, row in df.iterrows():
        s = row['staff_id']
        d = row['date']
        h = row['hour']
    
        cost[s, d, h] = row['salary']        # 1時間あたりのコスト
        max_cost[d, h] = row['max_cost']    # 時間帯ごとの最大人件費
        pred_sales[d, h] = row['pred_sale_per_hour']  # 


In [77]:
for d, h in max_cost:
    model.Add(
        sum(
            cost[s, d, h] * w
            for (s, dd, hh), w in work.items()
            if dd == d and hh == h
        ) <= max_cost[d, h]
    )
